In [1]:
from site_data_utils.site_analysis import get_info

import osmnx as ox

location = (39.3158421, -102.8736196)

from pygris.geocode import geocode
from Existing_Site_Data.geocode_sites import loc_data


/Users/sampackman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sampackman/Documents/ev_charging_analysis/site_data_utils/locational_data/get_data.py:5: DtypeWarning: Columns (18,26,70,72,85,131) have mixed types. Specify dtype option on import or set low_memory=False.
  j40_data = read_csv('site_data_utils/locational_data/Justice40/j40_data.csv')


Site Data found for 550/550 locations


In [2]:
address =  '17507 CO-59, Seibert, CO'

# ox.geocoder.geocode(address)

In [3]:
from pygris.geocode import geocode as gr_geocode
from osmnx.geocoder import geocode as ox_geocode
ox_geocode(address)


,longitude,latitude,GEOID,address
0,-102.874267,39.29131,080639624003156,"17507 CO-59, Seibert, CO"


In [5]:
from math import isnan
isnan(loc_data[loc_data.Address == '1871 Shawano Ave, Green Bay, WI 54303-2633'].Latitude)

/var/folders/gq/3wmh4pnj2rl1rnld2znxz3cc0000gn/T/ipykernel_85219/2883155737.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  isnan(loc_data[loc_data.Address == '1871 Shawano Ave, Green Bay, WI 54303-2633'].Latitude)


True

In [6]:
# G = ox.graph_from_point(center_point = location, dist = 500, network_type= 'walk')


In [10]:
# location = (gr_geocode(address).latitude,  gr_geocode(address).longitude)
print(get_info(address, output = "DataFrame", debug = True))



/Users/sampackman/Library/Python/3.9/lib/python/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]
/Users/sampackman/Library/Python/3.9/lib/python/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]
/Users/sampackman/Library/Python/3.9/lib/python/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(c) for c in o]
/Users/sampackman/Library/Python/3.9/lib/python/site-packages/shapely/geometry/polygon.py:91: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in th

Income of surrounding areas                      47386.0
Estimated cost of transportation                 11811.0
Cost burden for transportation                  0.225332
Proportion of households without vehicles            2.8
Walk times to…Adult Education                2219.284222
                                                ...     
subway stations                                        0
Number of Stations                                     0
ev_dc_fast_num                                         0
ev_level1_evse_num                                     0
ev_level2_evse_num                                     0
Length: 183, dtype: object


In [ ]:
import pandas as pd

d  = pd.DataFrame({
    'a': [1,2,3], 'b': [4,5,6]
})

In [ ]:
for i in d:
    print(i)

a
b
